In [8]:
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun

In [33]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=2,doc_content_chars_max=500)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv,description="Query arxiv papers")


In [34]:
api_wrapper_wikipedia = WikipediaAPIWrapper(top_k_results=2, doc_content_chars_max=500)
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper_wikipedia, description="Query Wikipedia articles")


In [35]:
from dotenv import load_dotenv
load_dotenv()

import os

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [36]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily = TavilySearchResults()

In [15]:
tools=[arxiv,wikipedia,tavily]

In [16]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="qwen-qwq-32b")

In [18]:
llm_with_tools = llm.bind_tools(tools=tools)

In [20]:
from typing_extensions import TypedDict
from langchain_core.messages import  AnyMessage
from typing import Annotated
from langgraph.graph.message import add_messages


In [21]:
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    

In [22]:
from IPython.display import display,Image
from langgraph.graph import StateGraph,START,END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition




In [31]:
def tool_calling_llm(state:State):
    return{"messages":[llm_with_tools.invoke(state["messages"])]}
    

builder = StateGraph(State)
builder.add_node("tool_calling_llm",tool_calling_llm)
builder.add_node("tools",ToolNode(tools))

builder.add_edge(START,"tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    tools_condition,
)
builder.add_edge("tools",END)

graph = builder.compile()




